# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 50
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast
import distributionAlgorithmForecastPolling

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm, minute_interval)

#distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
  #                  bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)



Minute:  480
Waiting BEVs:  [11, 14, 31, 32, 34, 36, 39, 49]
Update fueled solar energy
Charging BEVs:  []


Minute:  495
Waiting BEVs:  [11, 14, 31, 32, 34, 36, 39, 49]
Add charging BEVs
Update fueled solar energy
3.25 charging_power_per_bev
0 charging time
Update Charging energy of BEVs
Charging BEV Liste vor residual berechnung:  [11]
Charging BEVs:  [11]


Minute:  510
Waiting BEVs:  [14, 31, 32, 34, 36, 39, 49, 2, 4, 6, 8, 9, 15, 16, 22, 25, 26, 27, 30, 33, 35, 38, 43, 45]
Update fueled solar energy
8.5 charging_power_per_bev
15 charging time
Update Charging energy of BEVs
Charging BEV Liste vor residual berechnung:  [11]
Die Restladezeit des BEVS 11 endet vor dem nächsten Interval
Charging BEVs:  [14]


Minute:  525
Waiting BEVs:  [31, 32, 34, 36, 39, 49, 2, 4, 6, 8, 9, 15, 16, 22, 25, 26, 27, 30, 33, 35, 38, 43, 45]
Add charging BEVs
Update fueled solar energy
6.875 charging_power_per_bev
7.267058823529396 charging time
Update Charging energy of BEVs which came between interva

Charging BEVs:  [6, 8, 9, 15, 16]


Minute:  675
Waiting BEVs:  [22, 25, 26, 27, 30, 33, 35, 38, 43, 45, 1, 5, 7, 12, 13, 21, 28, 44, 46, 47, 23]
Update fueled solar energy
10.25 charging_power_per_bev
86.99411764705883 charging time
Update Charging energy of BEVs which came between intervals
65.4588235294118 charging time
Update Charging energy of BEVs which came between intervals
47.04236453201975 charging time
Update Charging energy of BEVs which came between intervals
17.68955223880596 charging time
Update Charging energy of BEVs which came between intervals
3.9360000000000355 charging time
Update Charging energy of BEVs which came between intervals
Charging BEV Liste vor residual berechnung:  [6, 8, 9, 15, 16]
Die Restladezeit des BEVS 6 endet vor dem nächsten Interval
Charging BEV Liste vor residual berechnung:  [8, 9, 15, 16, 22]
Charging BEV Liste vor residual berechnung:  [8, 9, 15, 16, 22]
Charging BEV Liste vor residual berechnung:  [8, 9, 15, 16, 22]
Charging BEV Liste vor 

3.104000000000042 charging time
Update Charging energy of BEVs which came between intervals
Charging BEV Liste vor residual berechnung:  [9, 15, 22, 26, 30]
Charging BEV Liste vor residual berechnung:  [9, 15, 22, 26, 30]
Charging BEV Liste vor residual berechnung:  [9, 15, 22, 26, 30]
Charging BEV Liste vor residual berechnung:  [9, 15, 22, 26, 30]
Charging BEV Liste vor residual berechnung:  [9, 15, 22, 26, 30]
Charging BEVs:  [9, 15, 22, 26, 30]


Minute:  825
Waiting BEVs:  [38, 45, 7, 13, 21, 28, 44, 46, 47, 23, 29, 10, 17, 19, 37, 48, 0, 3, 18, 20, 24, 41, 42, 40]
Update fueled solar energy
8.1 charging_power_per_bev
197.04236453201975 charging time
Update Charging energy of BEVs which came between intervals
167.68955223880596 charging time
Update Charging energy of BEVs which came between intervals
168.0 charging time
Update Charging energy of BEVs which came between intervals
76.82089552238801 charging time
Update Charging energy of BEVs which came between intervals
18.10400000

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 21 von 50<br>Unterbrochene Aufladevorgänge 0<br>Aufgeladene Solarenergie insgesamt: 102.99 kWh<br>Ungenutzte Solarenergie insgesamt: 0 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)